In the conclusion of my <a href="../tokenizer-problems/tokenizer-problems.html" target="_blank">recent blog post</a> I argued that I disagree with Andrej Karpathy's claims about the current state of tokenizer availability and tooling. In particular:

- I do not agree that there are no good alternatives to OpenAI's GPT-4 tokenizer and their `tiktoken` library, especially since the <a href="https://ai.meta.com/blog/meta-llama-3/" target="_blank">release of Llama 3</a>.
- Also, it is not the case that there are no good tools to train tokenizers from scratch fast, as the `tokenizers` library from Hugging Face has the functionality for both training and running tokenizers in inference mode. It is built in Rust, so it is extremely fast, and has reasonably approachable documentation.

This prompted me to do a deep dive into tokenizers and how one would go about building one from scratch. Inspired by the <a href="https://www.fast.ai/posts/2016-10-08-teaching-philosophy.html#good-education-starts-with-the-whole-game" target="_blank">top-down approach</a> from Jeremy Howard, whose courses I enjoy a lot, this blog post will start with the very basics of tokenizers and then focus on how to train a Llama 3-like tokenizer on your own data with as little code as possible. Finally, we will explore what influence different proportions of English/non-English/code data have on the final vocabulary learnt by the tokenizer, as well as discuss a paper on how tokenizer design choices impact the downstream performance of the LLM.

In the second part, we will explore what influence different proportions of English/non-English/code data have on the final vocabulary learnt by the tokenizer, as well as discuss a paper on how tokenizer design choices impact the downstream performance of the LLM, so stay tuned! Now, let's jump right in!.

# Warm-up: basics of tokenization

I am mostly assuming that if you found and opened this article, you have at least a basic understanding of what tokenization is and what its purpose is, but let's do a quick revision just to make sure we are all on the same page.

**Why do we need tokenization in the first place?**

1. Majority of machine learning models operate on numbers, and (large) language models are no exception.
2. Whether it is natural language, code, or something else, we want to be able to input strings into the language models.
3. This is where tokenization comes in, it is a **process of converting strings of text into numbers** that can then be fed into language models.

:::{.callout-note}
This last point is actually the reason why the current state-of-the-art language models are not actually end-to-end systems.
:::

**How do tokenizers work on a high level?**

1. First, the long input string is split into smaller chunks called *tokens*.
1. Then, a *vocabulary*, which is a mapping from known tokens to integers, is used to convert the tokens to integer ids.
1. These ids can be fed as input into language models.

**What are some qualities that we want our tokenizers to have?**

We want our tokenizers to be able to process various kinds of inputs. In this article, we will not be able to fully dig into the various design choices made when building tokenizers to achieve this, but here is a (likely non-exhaustive) list of qualities that we want the tokenizers to have:

1. They should work on both English and non-English texts, both uppercase and lowercase, both Latin and non-Latin alphabet.
1. They should be able to handle various kinds of special characters that we might find in the wild, such as emojis 🤗.
1. They should be able to handle code.
1. (Optional) They should be able to tokenize unseen words without the need to include a special `<unk>` token in their vocabulary.

**How does this look in code?**

Before diving into training a tokenizer, let's illustrate some of the aspects described above in code. We will find that all of the above is very simple to do by using the `tokenizers` library!

In [5]:
#|code-fold: true
#|output: false
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
import json
from pprint import pprint
from tokenizers import (
    Tokenizer, Regex, models, pre_tokenizers, processors, decoders
)
from transformers import AutoTokenizer

In [7]:
#|code-fold: true
#|output: false

# This code will only be used later when training a tokenizer
from datasets import load_dataset

dataset = load_dataset("wikitext", "wikitext-103-raw-v1", split="train")
len(dataset)

1801350

In [8]:
#|output: false
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Now that we have a tokenizer defined, let's see how we can tokenizer a piece of text:

In [9]:
text = "Some text to be tokenized"
tokens = tokenizer.tokenize(text)
tokens

['Some', 'Ġtext', 'Ġto', 'Ġbe', 'Ġtoken', 'ized']

We can see that the string was split into smaller tokens. The `Ġ` is how the `Llama 3` tokenizer represents spaces in its vocabulary, so you can actually think of the tokens as ` text`, ` to`, etc.

:::{.callout-note}
It turns out that it is very likely that nowadays the use of the `Ġ` Unicode character is a historical artifact that started with the GPT-2 implementation. A very strong evidence for this claim is that the `GPT-4` tokenizer has ordinary spaces in its vocabulary. See @sec-next-steps for more.
:::

Next, let's turn these tokens into ids:

In [10]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)
token_ids

[8538, 1495, 311, 387, 4037, 1534]

As simple as that! We could now feed these `token_ids` into a language model, though it is important to note that in practice we would perform everything in one step:

In [11]:
inputs = tokenizer(text, return_tensors="pt")
pprint(inputs, sort_dicts=False)  # pprint stands for "pretty print"

{'input_ids': tensor([[128000,   8538,   1495,    311,    387,   4037,   1534]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}


We would then call a Hugging Face-compatible model like so:
```python
# Define the model before this line
outputs = model(**inputs)
```

:::{.callout-note}
For the sake of completeness, it is worth noting that one would only tokenize a single string on its own during inference. During training, one would either tokenize the whole dataset in batches before training or tokenize a batch of data on the fly. The former is usually more reliable and allows to restart training more easily if a crash occurs (see Thomas Wolf's <a href="https://www.youtube.com/watch?v=2-SPH9hIKT8" target="_blank">video</a>).
:::

# Training a Llama 3 tokenizer

This section discusses the code needed to train a new tokenizer from an old one using the `tokenizers` library. For now, we will assume that we already have a dataset to train on, and we will simply reuse the architecture of `Llama 3` tokenizer, so that we do not have to worry about the different tokenizer design choices.

First, we define a function that takes a Hugging Face datasets and return a batched iterator over it:

In [12]:
# Define an iterator over the training split of the dataset
def batch_iterator(dataset, batch_size=1000, verbose=False):
    if verbose:
        print(f"Dataset size: {len(dataset)}")

    for i in range(0, len(dataset), batch_size):
        yield dataset[i:i+batch_size]["text"]


With that, we can go ahead and use it to train a new tokenizer:

In [30]:
#|output: false
new_tokenizer = tokenizer.train_new_from_iterator(
    batch_iterator(dataset, verbose=False),
    len(tokenizer.get_vocab()),
    initial_alphabet=pre_tokenizers.ByteLevel.alphabet(),
)

In [33]:
new_tokenizer.add_tokens(list(tokenizer.get_added_vocab().keys()))

print(f"Vocab length={len(new_tokenizer.get_vocab())}")

# Save the new tokenizer
new_tokenizer.save_pretrained("new-llama-tokenizer-english-only");

Vocab length=128256


This takes some time (about 1 minute on my machine), but the result is a working tokenizer, as easy as that!

## Assembling a Llama 3 tokenizer ourselves

Let's now dive a bit deeper and see how we could construct the tokenizer ourselves. This is where the different design decisions come into play.

Essentially, there are up to five components (most of them being optional) that have to be specified to have a working tokenizer. We will cover them briefly here, but I encourage reading <a href="https://huggingface.co/docs/tokenizers/en/components" target="_blank">this</a> great piece of documentation to learn more about the various components available in the `tokenizers` library that can be used to assemble a tokenizer.

The 5 components are:

1. **Normalizer** (optional): pre-process the input string for a given use case, e.g. strip accents (`é -> e`) or make the strings lowercase.
1. **Pre-tokenizer** (optional): performs some initial splitting with the main intent being to prevent the existence of tokens that are too long, e.g. ones that connect multiple common words. We could avoid this by having a pre-tokenizer that splits across spaces.
1. **Model**: an algorithm that performs the tokenization (i.e. takes strings, splits them an converts into tokens).
1. **Post-processor** (optional): used for post-processing the tokenized string, e.g. we could add special tokens or apply some other template.
1. **Decoder** (optional): helps using the tokenizer in the opposite directiom. i.e. mapping a list of token ids into readable text. In the simplest case, just the model's vocabulary is enough to perform the reversal, but certain normalizers or pre-tokenizers add special characters which are removed by the decoder.

Let's now build our own `Llama 3` tokenizer! We can check <a href="https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/blob/main/tokenizer.json" target="_blank">this</a> file to learn what the hyperparamaters are and just copy them over. <font color="red">It turns out that Llama 3 uses...</font> These are the design choices that have to be made if we were to build a *completely* new tokenizer from scratch, <font color="red">but it turns out that, in practice, it is enough to simply use a BPE model and only tune dataset used for training, the vocabulary size and the regular expression used to split strings during pre-tokenization.</font>

In [26]:
tokenizer_custom = Tokenizer(models.BPE(ignore_merges=True))
tokenizer_custom.pre_tokenizer = pre_tokenizers.Sequence([
    pre_tokenizers.Split(
        pattern=Regex(
            "(?i:'s|'t|'re|'ve|'m|'ll|'d)|[^\\r\\n\\p{L}\\p{N}]?\\p{L}+|\\p{N}{1,3}| ?[^\\s\\p{L}\\p{N}]+[\\r\\n]*|\\s*[\\r\\n]+|\\s+(?!\\S)|\\s+"
        ),
        behavior="isolated",
        invert=False,
    ),
    pre_tokenizers.ByteLevel(
        add_prefix_space=False, trim_offsets=True, use_regex=False
    )
])
# tokenizer_custom.pre_tokenizer = reference_tokenizer._tokenizer.pre_tokenizer
tokenizer_custom.post_processor = processors.Sequence([
    processors.ByteLevel(
        add_prefix_space=True, trim_offsets=False, use_regex=True
    ),
    processors.TemplateProcessing(
        single="<|begin_of_text|> $A",
        pair="<|begin_of_text|> $A <|begin_of_text|>:1 $B:1",
        special_tokens=[("<|begin_of_text|>", 128000)],
    ),
])
tokenizer_custom.decoder = decoders.ByteLevel()

In [27]:
#|output: false
from tokenizers import trainers

trainer = trainers.BpeTrainer(
    vocab_size=len(tokenizer.get_vocab()) - len(pre_tokenizers.ByteLevel.alphabet()),
    initial_alphabet=pre_tokenizers.ByteLevel.alphabet()
)

tokenizer_custom.train_from_iterator(
    batch_iterator(dataset, verbose=False), trainer=trainer
)

In [83]:
tokenizer_custom.add_tokens(list(tokenizer.get_added_vocab().keys()))

print(f"Vocab length={len(tokenizer_custom.get_vocab())}")

# Save the new tokenizer
tokenizer_custom.save("new-llama-tokenizer-custom-english-only/tokenizer.json")

Vocab length=128256


In [85]:
with open("new-llama-tokenizer-english-only/tokenizer.json", "r") as f:
    new_tokenizer_config = json.load(f)

with open("new-llama-tokenizer-custom-english-only/tokenizer.json", "r") as f:
    custom_tokenizer_config = json.load(f)

custom_tokenizer_config["model"]["merges"] == new_tokenizer_config["model"]["merges"]

True

# Final notes

1. Normalization is not used to make tokenization process reversible

# Next steps (for the curious reader) {#sec-next-steps}

I leave here some questions arose while writing this article that I could not find simple answers to. I hope to come back to them at some point in the future, but if you end up going down the rabbit holes to find the answers, I would be curious to know the answers! Here is the list:

1. Why are spaces replaced with the `Ġ` special character in BPE-based tokenizer? The same Thomas Wolf <a href="https://discuss.huggingface.co/t/bpe-tokenizers-and-spaces-before-words/475" target="_blank">argues</a> that this was done do avoid digesting spaces which are used in the standard BPE algorithm. However, it is unclear whether we could get by without doing this nowadays. Answering this question would probably require diving deeper into how the BPE algorithm works. Sounds like a fun direction to explore in the future!
    - <font color="red">Another source <a href="https://github.com/openai/gpt-2/blob/master/src/encoder.py#L9" target="_blank">on G dot</a></font>
1. The <a href="https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/raw/main/tokenizer.json" target="_blank">`Llama-3-8B-Instruct`</a> and <a href="https://huggingface.co/meta-llama/Meta-Llama-3-70B-Instruct/raw/main/tokenizer.json" target="_blank">`Llama-3-70B-Instruct`</a> tokenizers are almost identical apart from one setting: the former has the `model.ignore_merges` key specified as `true`, while the latter does not have such a key specified. The question is simple, what difference does this make? A good starting point to answer this question could be <a href="https://github.com/huggingface/tokenizers/blob/71c2a8d01a56cd7bd28148c309e210c47dac78e7/tokenizers/src/models/bpe/model.rs#L466" target="_blank">this</a> piece of code.
1. What are some metrics to track to evaluate the quality of a trained tokenizer?
1. <font color="red">The regex used for splitting is not ideal (unicode aposthrophe)</font>
1. <font color="red">Original BPE: https://github.com/rsennrich/subword-nmt and https://aclanthology.org/P16-1162.pdf</font>
1. <font color="red">FastBPE https://github.com/Yikai-Liao/efficient_bpe</font>

# Conclusion



# Sources

1. Let's build the GPT Tokenizer (<a href="https://www.youtube.com/watch?v=zduSFxRajkE" target="_blank">video</a>) by Andrej Karpathy.
1. A little guide to building Large Language Models in 2024 (<a href="https://www.youtube.com/watch?v=2-SPH9hIKT8" target="_blank">video</a>) by Thomas Wolf.
1. Training a new tokenizer from an old one (<a href="https://huggingface.co/learn/nlp-course/en/chapter6/2" target="_blank">article</a>) on the Hugging Face course.
1. Another Implementation (faster and more effecient) of BPE Training Algorithm (<a href="https://github.com/huggingface/tokenizers/issues/1400" target="_blank">GitHub issue</a>)
1. Getting the most out of your tokenizer for pre-training and domain adaptation (<a href="https://arxiv.org/pdf/2402.01035.pdf" target="_blank">arXiv PDF</a>)